In [2]:
import sys
import os

# Add the parent directory to the system path
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

from tqdm import tqdm
import src.network as network
import src.utils
import os
import argparse
import numpy as np
import json
import torch
import torch.nn as nn
from torch.utils import data
from datasets.kitti_360 import KittiDatasetLoader
from datasets.cityscapes_v1 import CityscapesDatasetLoader
from src.metrics import StreamSegMetrics
from reportlab.lib.pagesizes import landscape, letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib import colors


NameError: name '__file__' is not defined

In [31]:
import numpy as np 
import os 
import json


In [38]:
with open('../assets/rank_1_val.json') as f:
    master_val = json.load(f)

In [39]:
# sort the master val json by the emd_block_11 value

master_val = sorted(master_val, key=lambda x: x['emd_block_11'])

In [34]:
# in all the rows, replace 'image_path' with 'image'
for i in range(len(master_val)):
    master_val[i]['image'] = master_val[i]['image_path']
    del master_val[i]['image_path']

In [35]:
# image looks like this: "image": "datasets/data/KITTI-360/data_2d_raw/2013_05_28_drive_0000_sync/image_00/data_rect/0000000386.png",
# add ground truth like this for each image like this:  "ground_truth": "datasets/data/KITTI-360/data_2d_semantics/train/2013_05_28_drive_0000_sync/image_00/semantic/0000000386.png",

for i in range(len(master_val)):
    master_val[i]['ground_truth'] = master_val[i]['image'].replace('data_2d_raw', 'data_2d_semantics/train').replace('data_rect', 'semantic')
    master_val[i]['ground_truth'] = master_val[i]['ground_truth'].replace('image_00', 'image_00')
    master_val[i]['ground_truth'] = master_val[i]['ground_truth'].replace('png', 'png')
    master_val[i]['ground_truth'] = master_val[i]['ground_truth'].replace('jpg', 'png')

In [36]:
# Add these two rows to each entry in the master_val json
# "image_exists": true,
#"ground_truth_exists": true

for i in range(len(master_val)):
    master_val[i]['image_exists'] = True
    master_val[i]['ground_truth_exists'] = True


In [40]:
print(len(master_val))

# select first 2000 elements and split them into 4 groups of 500 each

group1 = master_val[:500]
group2 = master_val[500:1000]
group3 = master_val[1000:1500]
group4 = master_val[1500:2000]


# print mean and std of emd_block_11 for each group

group1_emd = [x['emd_block_11'] for x in group1]
group2_emd = [x['emd_block_11'] for x in group2]
group3_emd = [x['emd_block_11'] for x in group3]
group4_emd = [x['emd_block_11'] for x in group4]

print('Group 1 mean:', np.mean(group1_emd))
print('Group 1 std:', np.std(group1_emd))

print('Group 2 mean:', np.mean(group2_emd))
print('Group 2 std:', np.std(group2_emd))

print('Group 3 mean:', np.mean(group3_emd))
print('Group 3 std:', np.std(group3_emd))

print('Group 4 mean:', np.mean(group4_emd))
print('Group 4 std:', np.std(group4_emd))




# save the groups into separate json files

# with open('../assets/group1.json', 'w') as f:
#     json.dump(group1, f, indent=4)

# with open('../assets/group2.json', 'w') as f:
#     json.dump(group2, f, indent=4)

# with open('../assets/group3.json', 'w') as f:
#     json.dump(group3, f, indent=4)

# with open('../assets/group4.json', 'w') as f:
#     json.dump(group4, f, indent=4)

# print('Done')

2616
Group 1 mean: 18.353369822968112
Group 1 std: 0.5259146374729137
Group 2 mean: 19.337871654888623
Group 2 std: 0.18687033691444538
Group 3 mean: 19.83987479966404
Group 3 std: 0.11170702167243415
Group 4 mean: 20.164787628829757
Group 4 std: 0.08096239178264612


In [43]:
 python ./tests/test_v5.py --model deeplabv3plus_resnet101 --gpu_id 0 --checkpoint_dir checkpoints/ --json_file1 ./assets/group1.json --json_file2 ./assets/group1.json --num_test 500
 python ./tests/test_v5.py --model deeplabv3plus_resnet101 --gpu_id 0 --checkpoint_dir checkpoints/ --json_file1 ./assets/group1.json --json_file2 ./assets/group2.json --num_test 500
 python ./tests/test_v5.py --model deeplabv3plus_resnet101 --gpu_id 0 --checkpoint_dir checkpoints/ --json_file1 ./assets/group1.json --json_file2 ./assets/group3.json --num_test 500
 python ./tests/test_v5.py --model deeplabv3plus_resnet101 --gpu_id 0 --checkpoint_dir checkpoints/ --json_file1 ./assets/group1.json --json_file2 ./assets/group4.json --num_test 500

Device: cuda
setting up metrics
Model: deeplabv3plus_resnet101, Output Stride: 16
Model restored from ../checkpoints/latest_bucket_0_rand_deeplabv3plus_resnet101_kitti_os16.pth
Number of validation images for kitti_360: 500
Number of validation ground truth images for kitti_360: 500



0it [00:00, ?it/s]
0it [00:00, ?it/s]
Traceback (most recent call last):
  File "test_v5.py", line 194, in <module>
    main()
  File "test_v5.py", line 184, in main
    process_dataset(opts.json_file2, 'kitti_360', opts, model, device, metrics, results)
  File "test_v5.py", line 131, in process_dataset
    val_score = validate(opts=opts, model=model, loader=val_loader, device=device, metrics=metrics, image_paths=val_image_paths, label_paths=val_ground_truth_paths)
  File "test_v5.py", line 40, in validate
    for i, (images, labels) in tqdm(enumerate(loader)):
  File "c:\Users\kbpra\anaconda3\envs\lab\lib\site-packages\tqdm\std.py", line 1181, in __iter__
    for obj in iterable:
  File "c:\Users\kbpra\anaconda3\envs\lab\lib\site-packages\torch\utils\data\dataloader.py", line 631, in __next__
    data = self._next_data()
  File "c:\Users\kbpra\anaconda3\envs\lab\lib\site-packages\torch\utils\data\dataloader.py", line 1346, in _next_data
    return self._process_data(data)
  File "c:\

^C



0it [00:00, ?it/s]
0it [00:00, ?it/s]
Traceback (most recent call last):
  File "test_v5.py", line 194, in <module>
    main()
  File "test_v5.py", line 184, in main
    process_dataset(opts.json_file2, 'kitti_360', opts, model, device, metrics, results)
  File "test_v5.py", line 131, in process_dataset
    val_score = validate(opts=opts, model=model, loader=val_loader, device=device, metrics=metrics, image_paths=val_image_paths, label_paths=val_ground_truth_paths)
  File "test_v5.py", line 40, in validate
    for i, (images, labels) in tqdm(enumerate(loader)):
  File "c:\Users\kbpra\anaconda3\envs\lab\lib\site-packages\tqdm\std.py", line 1181, in __iter__
    for obj in iterable:
  File "c:\Users\kbpra\anaconda3\envs\lab\lib\site-packages\torch\utils\data\dataloader.py", line 631, in __next__
    data = self._next_data()
  File "c:\Users\kbpra\anaconda3\envs\lab\lib\site-packages\torch\utils\data\dataloader.py", line 1346, in _next_data
    return self._process_data(data)
  File "c:\

Device: cuda
setting up metrics
Model: deeplabv3plus_resnet101, Output Stride: 16
Model restored from ../checkpoints/latest_bucket_0_rand_deeplabv3plus_resnet101_kitti_os16.pth
Number of validation images for kitti_360: 500
Number of validation ground truth images for kitti_360: 500
Device: cuda
setting up metrics
Model: deeplabv3plus_resnet101, Output Stride: 16
Model restored from ../checkpoints/latest_bucket_0_rand_deeplabv3plus_resnet101_kitti_os16.pth
Number of validation images for kitti_360: 500
Number of validation ground truth images for kitti_360: 500



0it [00:00, ?it/s]
0it [00:00, ?it/s]
Traceback (most recent call last):
  File "test_v5.py", line 194, in <module>
    main()
  File "test_v5.py", line 184, in main
    process_dataset(opts.json_file2, 'kitti_360', opts, model, device, metrics, results)
  File "test_v5.py", line 131, in process_dataset
    val_score = validate(opts=opts, model=model, loader=val_loader, device=device, metrics=metrics, image_paths=val_image_paths, label_paths=val_ground_truth_paths)
  File "test_v5.py", line 40, in validate
    for i, (images, labels) in tqdm(enumerate(loader)):
  File "c:\Users\kbpra\anaconda3\envs\lab\lib\site-packages\tqdm\std.py", line 1181, in __iter__
    for obj in iterable:
  File "c:\Users\kbpra\anaconda3\envs\lab\lib\site-packages\torch\utils\data\dataloader.py", line 631, in __next__
    data = self._next_data()
  File "c:\Users\kbpra\anaconda3\envs\lab\lib\site-packages\torch\utils\data\dataloader.py", line 1346, in _next_data
    return self._process_data(data)
  File "c:\

Device: cuda
setting up metrics
Model: deeplabv3plus_resnet101, Output Stride: 16
Model restored from ../checkpoints/latest_bucket_0_rand_deeplabv3plus_resnet101_kitti_os16.pth
Number of validation images for kitti_360: 500
Number of validation ground truth images for kitti_360: 500



0it [00:00, ?it/s]
0it [00:00, ?it/s]
Traceback (most recent call last):
  File "test_v5.py", line 194, in <module>
    main()
  File "test_v5.py", line 184, in main
    process_dataset(opts.json_file2, 'kitti_360', opts, model, device, metrics, results)
  File "test_v5.py", line 131, in process_dataset
    val_score = validate(opts=opts, model=model, loader=val_loader, device=device, metrics=metrics, image_paths=val_image_paths, label_paths=val_ground_truth_paths)
  File "test_v5.py", line 40, in validate
    for i, (images, labels) in tqdm(enumerate(loader)):
  File "c:\Users\kbpra\anaconda3\envs\lab\lib\site-packages\tqdm\std.py", line 1181, in __iter__
    for obj in iterable:
  File "c:\Users\kbpra\anaconda3\envs\lab\lib\site-packages\torch\utils\data\dataloader.py", line 631, in __next__
    data = self._next_data()
  File "c:\Users\kbpra\anaconda3\envs\lab\lib\site-packages\torch\utils\data\dataloader.py", line 1346, in _next_data
    return self._process_data(data)
  File "c:\